# Basic sparse example

In [ ]:
import numpy as np
from scipy.linalg import circulant
from scipy.sparse import csr_matrix

In [ ]:
x = (np.arange(4) * 2 + 1).reshape((2, 2))
x = np.pad(x, 1, 'constant')
n_rows_in_x, n_columns_in_x = x.shape
x = x.reshape((-1))
n_elements_in_x = len(x)
x

In [ ]:
w = (np.arange(4) + 10).reshape((2, 2))
n_rows_in_w, n_columns_in_w = w.shape
w

In [ ]:
f = np.zeros((n_rows_in_x, n_columns_in_x))
f[:n_rows_in_w, :n_columns_in_w] = w
f = f.reshape(-1)
f

In [ ]:
w = circulant(f).T
w = csr_matrix(w)
w.toarray()

In [ ]:
stride = 1
n_rows_in_output = int(np.floor((n_rows_in_x - n_rows_in_w) / stride + 1))
n_columns_in_output = int(np.floor((n_columns_in_x - n_columns_in_w) / stride + 1))

indices = np.zeros(w.shape[0], dtype=bool)
for index_row, index_start in enumerate(range(0, indices.shape[0], n_columns_in_x)):
    if index_row >= n_rows_in_output:
        break
    index_end = index_start + n_columns_in_output
    indices[index_start:index_end] = [True] * n_columns_in_output

if n_rows_in_output * n_columns_in_output != np.sum(indices):
    raise Exception('Sum of indices should match the values in output')
indices

In [ ]:
w = w[indices, :]
w.toarray()

In [ ]:
goal = [[10, 11, 0, 0, 12, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 10, 11, 0, 0, 12, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 10, 11, 0, 0, 12, 13, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 10, 11, 0, 0, 12, 13, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 10, 11, 0, 0, 12, 13, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 10, 11, 0, 0, 12, 13, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 10, 11, 0, 0, 12, 13, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 11, 0, 0, 12, 13, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 11, 0, 0, 12, 13]]
np.sum(w.toarray() - goal != 0)

# Real data

In [ ]:
from typing import Optional
from timeit import default_timer

import numpy as np
from scipy import sparse, linalg
from scipy.sparse import csr_matrix
from tqdm import tqdm

In [ ]:
def load(x: str) -> np.ndarray:
    x = np.load('{}.npy'.format(x))
    x = x[0, 0, :, :]
    print(x.shape)
    return x

x = load('x')
y = load('y')
w = load('w')

x = np.pad(x, 1, 'constant')
n_rows_in_x, n_columns_in_x = x.shape
x = x.reshape((-1))
n_rows_in_w, n_columns_in_w = w.shape

In [ ]:
time_start = default_timer()
f = np.zeros((n_rows_in_x, n_columns_in_x))
f[:n_rows_in_w, :n_columns_in_w] = w
f = f.reshape(-1)
print(default_timer() - time_start)

In [ ]:
time_start = default_timer()
w = linalg.circulant(f).T
w = csr_matrix(w)
print(w.shape)
print(default_timer() - time_start)

In [ ]:
time_start = default_timer()
stride = 1
n_rows_in_output = int(np.floor((n_rows_in_x - n_rows_in_w) / stride + 1))
n_columns_in_output = int(np.floor((n_columns_in_x - n_columns_in_w) / stride + 1))

indices = np.zeros(w.shape[0], dtype=bool)
for index_row, index_start in enumerate(range(0, indices.shape[0], n_columns_in_x)):
    if index_row >= n_rows_in_output:
        break
    index_end = index_start + n_columns_in_output
    indices[index_start:index_end] = [True] * n_columns_in_output

if n_rows_in_output * n_columns_in_output != np.sum(indices):
    raise Exception('Sum of indices should match the values in output')
    
w = w[indices, :]
print(w.shape)
print(default_timer() - time_start)

In [ ]:
x = x.reshape((-1, 1))
y = y.reshape((-1, 1))

In [ ]:
time_start = default_timer()
original_w = w
refined_w = original_w.copy()

for i in tqdm(range(y.shape[0])):
    pass
    # w_tf, num_iter_tf = _trim_layer(X=X, y=Y[i, :], rho=5, alpha=1.8, lmbda=4)
    # refined_w[:, i] = w_tf

print('number of non-zero values in the original weight matrix = ', np.sum(original_w != 0))
print('number of non-zero values in the refined weight matrix = ', np.sum(refined_w != 0))
print(default_timer() - time_start)

In [ ]:
y_orig = y

In [ ]:
def load(x: str) -> np.ndarray:
    x = np.load('{}.npy'.format(x))
    x = x[0, 0, :, :]
    print(x.shape)
    return x

x = load('x')
y = load('y')

x = np.pad(x, 1, 'constant')
x = x.reshape((-1))

x = x.reshape((-1, 1))
y = y.reshape((-1, 1))

In [ ]:
y = y[0, :]

In [ ]:
time_start = default_timer()
rho = 5
alpha = 1.8
lmbda = 4

lmbda = 1 / lmbda
y = np.reshape(y, newshape=(1, -1))  # make sure y is a row vector
N = x.shape[0]

if y.shape[1] != x.shape[1]:
    raise ValueError("Dimensions of input data, X & y, are not consistent.")

print(default_timer() - time_start)

In [ ]:
time_start = default_timer()
Q = csr_matrix(x)
Q = Q @ Q.T
Q *= lmbda
print(default_timer() - time_start)

In [ ]:
time_start = default_timer()
q = -lmbda * x @ y.T
q = csr_matrix(1 / 2 + np.vstack([q, -q]))

P = csr_matrix((x.shape[0], 0))
P = P.T
P = sparse.hstack([P, -P])

c = csr_matrix((0, 1))
print(default_timer() - time_start)

In [ ]:
time_start = default_timer()
tmp = csr_matrix([[1, -1], [-1, 1]])
Q = sparse.kron(tmp, Q)
print(default_timer() - time_start)

In [ ]:
time_start = default_timer()
A = sparse.vstack([P, -sparse.eye(2 * N, 2 * N)])
c = sparse.vstack([c, csr_matrix((2 * N, 1))])
print(default_timer() - time_start)

In [ ]:
time_start = default_timer()
L = Q + rho * A.T @ A
L = csr_matrix(np.linalg.cholesky(L.toarray()))
U = L.T
print(default_timer() - time_start)

In [ ]:
time_start = default_timer()
z = csr_matrix((c.shape[0], 1))
x = csr_matrix((2 * N, 1))
u = csr_matrix((c.shape[0], 1))
print(default_timer() - time_start)

In [ ]:
time_start = default_timer()

print(default_timer() - time_start)

In [ ]:
time_start = default_timer()

print(default_timer() - time_start)

In [ ]:
time_start = default_timer()

print(default_timer() - time_start)

In [ ]:
lmbda = 4
lmbda = 1 / lmbda
y = np.reshape(y, newshape=(1, -1))  # make sure y is a row vector
N = X.shape[0]

if y.shape[1] != X.shape[1]:
    raise ValueError("Dimensions of input data, X & y, are not consistent.")

# Omega = np.where(y > 1e-6)[1]
Omega = list(range(y.shape[1]))
# Omega_c = np.where(y <= 1e-6)[1]
Omega_c = []

Q = lmbda * np.matmul(X[:, Omega], np.transpose(X[:, Omega]))
q = -lmbda * np.matmul(X, np.transpose(y))
P = X[:, Omega_c]
P = P.transpose()
c = np.zeros((len(Omega_c), 1))

Q = np.kron([[1, -1], [-1, 1]], Q)
q = 1 / 2 + np.append(q, -q, axis=0)
P = np.append(P, -P, axis=1)

A = np.append(P, -np.eye(2 * N, 2 * N), axis=0)
c = np.append(c, np.zeros((2 * N, 1)), axis=0)

# The ADMM part of the code
L = np.linalg.cholesky(Q + rho * np.matmul(A.transpose(), A))
U = L.transpose()

z = np.zeros((len(c), 1))
x = np.zeros((2 * N, 1))
u = np.zeros((len(c), 1))

L = torch.from_numpy(L)
U = torch.from_numpy(U)
A = torch.from_numpy(A)
q = torch.from_numpy(q)
c = torch.from_numpy(c)

z = torch.from_numpy(z)
x = torch.from_numpy(x)
u = torch.from_numpy(u)